In [1]:
!pip install langchain transformers torch torchvision torchaudio
!pip install transformers
!pip install torch
!pip install datasets
!pip install tiktoken
!pip install transformers_stream_generator
!pip install ollama
!pip install huggingface_hub
!pip install bitsandbytes
!pip install -U accelerate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: 

In [2]:
# Import Libraries
import tqdm
import pandas as pd
import transformers
from langchain.llms.base import LLM
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from transformers import pipeline
import torch
from langchain.schema import HumanMessage, AIMessage
import json
from typing import List, Union
from pydantic import Field
import ast
import re

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [67]:
# Is this notebook connected to GPU ?
print(torch.cuda.is_available())

# Is Gpu available in Backend ?
print(torch.backends.cudnn.enabled)

True
True


In [4]:
# Hugging Face API authentication
!huggingface-cli login --token hf_1234

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `LLM` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM`


In [109]:
# Load Hugging Face Model
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True,"device_map":"auto"},
    max_length=8192
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.25it/s]
Device set to use cuda:0


In [69]:
def generate_interview_questions(name, skills, responsibilities, job_title, job_description):
    prompt = f"""
    You are an AI specializing in generating basic structured interview questions. 
    Generate exactly 10 interview questions in Python **list format** (`["Q1", "Q2", "Q3", ...]`). 
    **Strictly follow the structure** below and return **only the list, without any explanations or extra text**.

    Generate:
    1. 3 HR-related questions (e.g., background, cuxlture fit, basic questions like coding skills, where do you see yourself in 5 years).
    2. 4 Technical questions based on skills, responsibilities, and job title.
    3. 3 Behavioral questions based on the job description.

    ### Candidate Information:
    - Name : {name}
    - Job Title: {job_title}
    - Skills: {skills}
    - Responsibilities: {responsibilities}

    ### Job Description:
    {job_description}

    ### **Output Format (Strictly Return in This Format)**
    ["Question 1", "Question 2", "Question 3", ..., "Question 10"]
    """
    response = pipeline(prompt,return_full_text=False)
    return response[0]["generated_text"]

In [101]:
# Load candidate data
data1 = pd.read_csv("./metadata/METADATA-FINAL.csv")
df = data1

#Clean Data
#df.fillna(0,inplace=True)
df = df[df["Name"] != "None"]
df.shape

(204, 11)

In [102]:
# Function to split DataFrame into N subsets of equal size
def split_dataframe(df, num_subsets=20):
    """
    Splits a DataFrame into smaller subsets of equal size.
    
    Parameters:
    df (DataFrame): The original DataFrame to split.
    num_subsets (int): Number of subsets to create.
    
    Returns:
    list of DataFrames: A list containing the split DataFrames.
    """
    subset_size = len(df) // num_subsets  # Calculate the number of rows per subset
    return [df.iloc[i * subset_size: (i + 1) * subset_size] for i in range(num_subsets)]

# Creating 20 subsets of 10 rows each
subsets = split_dataframe(df, num_subsets=20)

# Example: Accessing the first subset
# subset_1 = subsets[0]
# subset_2 = subsets[1]
# subset_3 = subsets[2]
# subset_4 = subsets[3]
# subset_5 = subsets[4]
# subset_6 = subsets[5]
# subset_7 = subsets[6]
# subset_8 = subsets[7]
# subset_9 = subsets[8]
# subset_10 = subsets[9]
# subset_11 = subsets[10]
# subset_12 = subsets[11]
# subset_13 = subsets[12]
# subset_14 = subsets[13]
# subset_15 = subsets[14]
# subset_16 = subsets[15]
# subset_17 = subsets[16]
# subset_18 = subsets[17]
subset_19 = subsets[18]
subset_20 = subsets[19]

In [108]:
subset_20

,file_name,resume_content,Name,Email,Phone,Location,Skills,Experience (Years),Responsibilities,Job Title,Education
268,Dhanalaxmi -BA.docx,Dhanalaxmi Sangole\nPh No: 916-282-9259\nEmail...,Dhanalaxmi Sangole,sam@vishconsultingservices.com,916-282-9259,"Madison, WI","['Business Process Engineering', 'Software Dev...",7+ years,['Developed IV&D on MMIS and Medicaid Eligibil...,"['Senior Business Analyst', 'Sr. Regulatory An...",[{'Degree': 'Bachelor of Computer Engineering'...
271,Diksha Bhati-Resume.pdf,Diksha Bhati\n(cid:131) +91 9354112277 # diksh...,Diksha Bhati,dikshab035@gmail.com,None,Delhi,"['Python', 'SQL', 'Keras', 'Scikit-Learn', 'Ma...",4 years,['Automated cron job scripts and integrated th...,"['Data Science Intern', 'Machine Learning Inte...",[{'Degree': 'Master of Science in Applied Math...
272,Divya Gajare Resume.pdf,Divya Gajare\nData Engineer\ndivs607@gmail.com...,Divya Gajare,divs607@gmail.com,+91 9131835943,Gwalior,"['SQL', 'Teradata', 'Greenplum', 'Power BI', '...",4 years,['Managing end-to-end requirement gathering fr...,"['Data Engineer', 'Senior Analyst', 'Graduate ...","[{'Degree': 'M. Tech in Information Security',..."
275,FARSHAD_K.pdf,"Farshad K\nLocation: Bengaluru, India\nTel: +9...",Farshad K,farshadabdulazeez@gmail.com,+91 81290 26217,Bengaluru,"['Statistics and Probability', 'Machine Learni...",1 year,"['Processing invoices, managing accounts payab...","['Accountant', 'Operations Manager']","[{'Degree': 'Bachelor of Commerce - Finance', ..."
276,Francis Gomes Resume.docx,Professional summary \n16 + years of experien...,W. Paul,w.paul@niit.com,+971 55029999,"Houston, Texas, United States","['Project Management', 'ITIL', 'Microsoft Cert...",16,['Developing analytical solutions for IT progr...,"['Business Strategy Analyst', 'PMO Consultant'...",[{'Degree': 'Post Graduate Diploma in Business...
278,GANESHJAGADEESAN.pdf,GANESH JAGADEESAN\njagadeesan.ganesh@gmail.com...,GANESH JAGADEESAN,jagadeesan.ganesh@gmail.com,+919884517404,Chennai,"['Data Science', 'Deep Learning', 'Solution De...",18,['Leading the Data science Delivery team of 20...,"['Data Science Lead', 'Sr. Enterprise Computin...","[{'Degree': 'MBA', 'University': 'upGrad Schoo..."
279,Gautami Bulusu Mobile Testing 3.docx,Gautami Bulusu\neGrove Systems Corporation\nCo...,Gautami Bulusu,krish@egrovesys.com,None,Hyderabad,"['Selenium WebDriver', 'Appium', 'TestNG', 'Ma...",Over 4 years,['Developed Test-Plans and Test-cases for Manu...,"['Software QA Test Engineer', 'QA Test Enginee...","[{'Degree': 'Bachelor of Technology, Electroni..."
281,Gopi.docx,Gopi\n ...,Gopi,gopijavafullstack@gmail.com,+1 732-630-7623,"Frisco, TX","['Java', 'Spring Boot', 'JUnit', 'Maven', 'RES...",8,"['Developing Java applications', 'Working on v...","['Senior Java Full Stack Developer', 'Sr. Java...","[{'Degree': 'Bachelor’s in Computers', 'Univer..."
282,Gowtham_Resume.pdf,Gowtham Ravichandran\nSoftware Developer\nrgow...,Gowtham Ravichandran,rgowtham2502@gmail.com,+91 9750257792,Bangalore,"['Data Analysis', 'SQL', 'Python', 'ASP.NET', ...",Total experience: 10 months (Sep 2023 - Dec 20...,['Design report dashboards for business perfor...,"['Data Analyst Intern', 'Product Engineer (RPA...","[{'Degree': 'B.Sc., Information Systems', 'Uni..."
283,Gururaja Murthy PMCPCSM.docx,Professional Summary ...,Gururaja Murthy,'gururaja.murthy@kolkata.iitkgp.ac.in','+91-9876543218',"Bangalore, India","['Project Management', 'Agile Scrum', 'QA Test...",18+ years,"['Project Management', 'Agile Scrum', 'QA Test...","['Project Manager', 'Agile Scrum Master', 'QA ...","[{'Degree': ""Master's in Computer Application""..."


In [110]:
df = subset_20

# Placeholder job description
job_description = """ 
JOB DESCRIPTION 

This Senior-Level Data Scientist role is ideal for the candidate with strong analytical and problem-solving skills, combined with deep expertise in applying advanced machine-learning techniques using big-data platforms and tools. An excellent verbal and written communicator, with ability to collaborate closely with business stakeholders across merchandising, marketing, and finance. This role will focus on optimizing pricing and promotional effectiveness to maximize sales, margin, customer engagement, and overall business performance. This is a hybrid role, reporting into the office in Denton Texas on Tuesdays and Weds of each week, working remotely the rest of the work week, so you must be located in the DFW area.

“At Sally Beauty Holdings, we find beauty in diversity. Our inclusivity and self-expression are what fuel our innovation and growth. You are welcome here, and you can thrive here. We find beauty in YOU. Join us.”

Primary Duties

Engage with business teams to find opportunities, gather requirements, and translate them into scalable technical solutions to deliver measurable business outcomes.
Lead the design and development of advanced data-science solutions, applying closed-loop testing and measurement methodologies to continuously optimize pricing and promotional effectiveness for maximum business impact.
Lead and develop machine learning models through all phases of development - design, training, evaluation, validation, and deployment - ensuring high performance and alignment with business goals.
Work closely with data engineers and platform architects to implement robust data pipelines, enabling real-time and batch decisioning solutions.
Monitor the health of operational and business metrics for solutions in production. Analyze trends and behavior patterns, and implement changes as needed to ensure sustained performance.
Present complex data-driven insights to business partners and executives. Translate technical results into business strategies and actionable recommendations that drive decision-making.
Research and apply new technologies and methodologies to continuously improve the team's technical capabilities, drive innovation, and foster a culture of learning.

Technical Skills

Knowledge, Skills & Abilities:

Strong programming skills in Python, R, or other similar languages, for data analysis and machine learning.
Proficient in SQL for querying large relational databases, with a solid understanding of database modeling and data warehousing principles.
Hands-on experience with MS Azure Databricks or other cloud-based data platforms.
Familiarity with Big Data tools like Spark, Hive etc.
Proven experience in building scalable data pipelines for analytics solutions.

Analysis And Modeling Skills

Advanced knowledge of machine learning techniques, including:
Supervised learning algorithms (e.g., XGBoost, CatBoost, Random Forests) for predictive modeling.
Unsupervised learning techniques (e.g., clustering/segmentation) and customer behavior analysis.
Experience in time-series forecasting, simulation, and optimization for demand prediction and promotional planning.
Knowledge of A/B testing, ANOVA/ANCOVA, and promo-mix modeling for measurement and evaluation.
Insights And Communication Skills

Ability to extract actionable business insights from complex data and identify the stories behind the patterns.
Excellent presentation skills, distilling complex analysis and concepts into concise business-focused takeaways.
Creativity to engineer novel features and signals, and to push beyond current tools and approaches.
Exceptional project management skills, with a focus on attention to detail and meeting deadlines.
Mentor junior data scientists and foster a culture of continuous learning within the team.

Qualifications

Master’s degree in Data Science, Machine Learning, Statistics, Computer Science, or a related field (PhD preferred).
5+ years of experience in data science, machine learning, or related fields.
Prior experience in retail analytics, particularly in pricing, promotions, or customer behavior analysis is highly preferred.

Competencies/Attributes

Takes ownership of results, maintaining high performance while balancing multiple complex tasks.
Communicates clearly and proactively with both technical and non-technical stakeholders.
Strong business acumen, with the ability to connect data insights to real-world business outcomes.
Detail-oriented and committed to continuous improvement.

Working Conditions /Physical Requirements

The work environment involves everyday risks or discomforts which require normal safety precautions typical of such places as offices, meeting and training rooms, retail stores, and residences or commercial vehicles, e.g., use of safe work practices with office equipment, avoidance of trips and falls, observance of fire regulations and traffic signals, etc. The work area is adequately lighted, heated, and ventilated.
The work is sedentary. Typically, the employee may sit comfortably to do the work. However, there may be some walking; standing; bending; carrying of light items such as papers, files, books, small parts; using a keyboard, driving an automobile, etc. No special physical demands are required to perform the work.

We offer a competitive salary and outstanding benefits package that includes medical, dental, vision, life Insurance, paid vacation and sick days, paid holidays, tuition reimbursement and 401(k) with company match. In addition, associates of SBH may take advantage of our in house salon with complementary services and a varied selection of food options at our corporate campus. Also, featured at our corporate campus, is both a Sally Beauty and CosmoProf Professional onsite store, where associates enjoy a great merchandise discount!
"""

# Store results in a list
generated_data = []

# Iterate through each subset and generate interview questions
for index, row in df.iterrows():
    name = row.get("Name", "Unknown")
    skills = row.get("Skills", "No Skills Listed")
    responsibilities = row.get("Responsibilities", "No Responsibilities Provided")
    job_title = row.get("Job Title", "No Job Title Provided")
    file_name = row.get("file_name", "Unknown")  # Get file_name for each entry

    # Generate questions for the candidate
    questions = generate_interview_questions(name, skills, responsibilities, job_title, job_description)

    # Store the result
    generated_data.append({"Name": name, "File Name": file_name, "Generated Questions": questions})

# Convert the list into a DataFrame
df_questions = pd.DataFrame(generated_data)

print("✅ Generated questions saved successfully!")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:12

✅ Generated questions saved successfully!


In [111]:
df_questions

,Name,File Name,Generated Questions
0,Dhanalaxmi Sangole,Dhanalaxmi -BA.docx,"\n\n\n\nHere is your output:\n\n[""What motivat..."
1,Diksha Bhati,Diksha Bhati-Resume.pdf,\n\n\n\nHere are the interview questions in Py...
2,Divya Gajare,Divya Gajare Resume.pdf,\n\n\n\nHere is the output in the required for...
3,Farshad K,FARSHAD_K.pdf,\n\n\n\nHere is the list of interview question...
4,W. Paul,Francis Gomes Resume.docx,\n\n\n\nHere are the 10 interview questions in...
5,GANESH JAGADEESAN,GANESHJAGADEESAN.pdf,\n\n\n\nHere are the interview questions in Py...
6,Gautami Bulusu,Gautami Bulusu Mobile Testing 3.docx,"\n\n\n\nHere is your output:\n\n[""What do you ..."
7,Gopi,Gopi.docx,\n\n\n\nHere are the interview questions in Py...
8,Gowtham Ravichandran,Gowtham_Resume.pdf,\n\n\n\nHere is your output in the required fo...
9,Gururaja Murthy,Gururaja Murthy PMCPCSM.docx,\n\n\n\nHere is the output in the required for...


In [112]:
import ast
import re

def extract_list_from_df(df, column_name="Generated Questions"):
    """
    Extracts the list of interview questions from a dataframe column.
    Returns a new dataframe with a cleaned list of questions.
    """
    def extract_list(text):
        try:
            # Ensure text is a string and find the first occurrence of a Python-style list
            match = re.search(r'\[.*?\]', str(text), re.DOTALL)
            if match:
                extracted_text = match.group(0)  # Extract the matched list
                return ast.literal_eval(extracted_text)  # Convert string to actual Python list
        except (SyntaxError, ValueError):
            return []  # Return an empty list if extraction fails
        
        return []  # Return empty if no match is found

    # Apply extraction function to the specified column
    df["Cleaned Questions"] = df[column_name].apply(extract_list)
    
    return df

# Apply the function
df_cleaned = extract_list_from_df(df_questions, column_name="Generated Questions")

# Display cleaned data
df_cleaned[["Name","File Name", "Cleaned Questions"]]

,Name,File Name,Cleaned Questions
0,Dhanalaxmi Sangole,Dhanalaxmi -BA.docx,[What motivated you to apply for this Senior B...
1,Diksha Bhati,Diksha Bhati-Resume.pdf,[What inspired you to pursue a career in data ...
2,Divya Gajare,Divya Gajare Resume.pdf,[What inspired you to pursue a career in data ...
3,Farshad K,FARSHAD_K.pdf,[What do you think makes you a strong fit for ...
4,W. Paul,Francis Gomes Resume.docx,[What motivated you to apply for this Senior-L...
5,GANESH JAGADEESAN,GANESHJAGADEESAN.pdf,[What do you think are the most significant ch...
6,Gautami Bulusu,Gautami Bulusu Mobile Testing 3.docx,[What do you know about the company Sally Beau...
7,Gopi,Gopi.docx,[What motivated you to apply for this Senior J...
8,Gowtham Ravichandran,Gowtham_Resume.pdf,[What do you think are the most important skil...
9,Gururaja Murthy,Gururaja Murthy PMCPCSM.docx,[What do you think are the most important skil...


In [113]:
df_cleaned.to_csv("Subset20_DS_Interview_Questions.csv",index=False)

In [114]:
import pandas as pd
import os

# Specify the directory where your CSV files are located
directory_path = 'Subsets/'

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)  # Read each CSV file into a DataFrame
        dataframes.append(df)  # Append the DataFrame to the list

# Concatenate all the dataframes in the list into one DataFrame
final_df = pd.concat(dataframes, ignore_index=True)

# Optionally, you can save the concatenated DataFrame to a new CSV file
final_df.to_csv('InterviewQuestions.csv', index=False)